<a href="https://colab.research.google.com/github/karandoshi1810/Machine-Learning/blob/main/KNN(on_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import statistics

In [81]:
#To import datafiles from Google Drive
from google.colab import drive

In [82]:
#To mount the drive data into colab cloud and enter the activation code
#4/1AY0e-g4UIdh51GMQrW4voFdq4EfqV0GfhKX-Fbnk1cHN4l6ZVJCsaLaMybY
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
cd /content/drive/"My Drive/Colab Notebooks/Dataset"

/content/drive/My Drive/Colab Notebooks/Dataset


In [84]:
#reading data from drive uing pandas library
df=pd.read_csv("insurance.csv")
print(df)
df.shape

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


(1338, 7)

In [85]:

df.describe()
median_insurance = df['charges'].median()
first_quantile = df['charges'].quantile(0.25)
second_quantile = df['charges'].quantile(0.50)
third_quantile = df['charges'].quantile(0.75)
print("Median:",median)
print("First Quantile:",first_quantile)
print("Second Quantile:",second_quantile)
print("Third Quantile",third_quantile)
#df = pd.get_dummies(df)
df.head

Median: 9382.033
First Quantile: 4740.28715
Second Quantile: 9382.033
Third Quantile 16639.912515


<bound method NDFrame.head of       age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]>

In [86]:
df['insurance_class']=[0 for i in range(len(df['charges']))]

for i in range(0,len(df['charges'])-1):
  if df['charges'][i]>first_quantile and df['charges'][i]<=second_quantile:
    df['insurance_class'][i]="Accepted with minor changes"
  elif df['charges'][i]>second_quantile and df['charges'][i]<=third_quantile:
    df['insurance_class'][i]="Accepted with major changes"
  elif df['charges'][i]>third_quantile:
    df['insurance_class'][i]="Rejected"
  else:
    df['insurance_class'][i]="Accepted"
df['insurance_class']
print(df)

#converting categorical data to numerical data
sex_code = {'male':1, 'female':2}
df['sex'] = df['sex'].map(sex_code)
smoker_code = {'yes':1, 'no':0}
df['smoker'] = df['smoker'].map(smoker_code)
region_code = {'southwest':1, 'southeast':2,'northeast':3,'northwest':4}
df['region'] = df['region'].map(region_code)

      age     sex     bmi  ...     region      charges              insurance_class
0      19  female  27.900  ...  southwest  16884.92400                     Rejected
1      18    male  33.770  ...  southeast   1725.55230                     Accepted
2      28    male  33.000  ...  southeast   4449.46200                     Accepted
3      33    male  22.705  ...  northwest  21984.47061                     Rejected
4      32    male  28.880  ...  northwest   3866.85520                     Accepted
...   ...     ...     ...  ...        ...          ...                          ...
1333   50    male  30.970  ...  northwest  10600.54830  Accepted with major changes
1334   18  female  31.920  ...  northeast   2205.98080                     Accepted
1335   18  female  36.850  ...  southeast   1629.83350                     Accepted
1336   21  female  25.800  ...  southwest   2007.94500                     Accepted
1337   61  female  29.070  ...  northwest  29141.36030                      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by Interactiv

In [87]:
num_neighbors=5

In [88]:
training = df.values[0:1338,0:7]
training[1]

array([18, 1, 33.77, 1, 0, 2, 1725.5523], dtype=object)

In [89]:
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column])

for i in range(len(training[0])):
  str_column_to_float(training,i)

training

array([[19.0, 2.0, 27.9, ..., 1.0, 1.0, 16884.924],
       [18.0, 1.0, 33.77, ..., 0.0, 2.0, 1725.5523],
       [28.0, 1.0, 33.0, ..., 0.0, 2.0, 4449.462],
       ...,
       [18.0, 2.0, 36.85, ..., 0.0, 2.0, 1629.8335],
       [21.0, 2.0, 25.8, ..., 0.0, 1.0, 2007.945],
       [61.0, 2.0, 29.07, ..., 1.0, 4.0, 29141.3603]], dtype=object)

In [90]:
trainingclass=df.values[0:1337,-1]
trainingclass

array(['Rejected', 'Accepted', 'Accepted', ..., 'Accepted', 'Accepted',
       'Accepted'], dtype=object)

In [91]:
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return (math.sqrt(distance))

In [92]:
#Last row of data as test data
testing=df.values[1337,0:7]
testing

array([61, 2, 29.07, 0, 1, 4, 29141.3603], dtype=object)

In [93]:
unique_list = []
for x in trainingclass:
  if x not in unique_list:
    unique_list.append(x)

print(unique_list)
for i in range(len(trainingclass)):
  for k in range(len(unique_list)):
    if(trainingclass[i]==unique_list[k]):
      trainingclass[i]=k
print(trainingclass)

['Rejected', 'Accepted', 'Accepted with minor changes', 'Accepted with major changes']
[0 1 1 ... 1 1 1]


In [95]:
#Distance of one point to test point
distance=[]
for i in range(len(training)-1):
  distance.append([euclidean_distance((training[i]),(testing)),trainingclass[i]])

distance

[[42.123258420972135, 0],
 [43.33693574769679, 1],
 [33.458106640992106, 1],
 [28.74914303070615, 0],
 [29.03508395028332, 1],
 [30.26696053454988, 1],
 [15.814452251026589, 2],
 [24.243945635972707, 2],
 [24.157350848137302, 2],
 [3.526031763895499, 0],
 [36.15414913948329, 1],
 [3.5676883271945163, 0],
 [38.51504770865538, 1],
 [12.064928512013655, 3],
 [36.49059604884524, 0],
 [42.37901485405247, 1],
 [9.32330949824149, 3],
 [38.396622572825336, 1],
 [12.732356419767706, 3],
 [31.777553398586242, 0],
 [7.148022453797975, 3],
 [31.354248515950754, 1],
 [43.36243650903395, 1],
 [27.186807462443987, 0],
 [24.147298503145233, 2],
 [4.45224662389675, 3],
 [6.466855882111491, 3],
 [7.3977716239419005, 3],
 [39.793708359488186, 1],
 [31.08493043260673, 0],
 [39.6691429199069, 0],
 [43.111367700410526, 1],
 [42.41722409587879, 1],
 [2.564683216305671, 3],
 [33.966585050605254, 0],
 [42.90379965690685, 1],
 [5.115762406523589, 3],
 [36.11637994040931, 1],
 [27.143323304267664, 0],
 [11.32646

In [96]:
distance.sort()
distance

[[1.605490579231159, 0],
 [1.9754746265138408, 3],
 [2.055632506067172, 0],
 [2.0796393918177256, 3],
 [2.304430515333452, 3],
 [2.316916053723138, 3],
 [2.466013990227955, 3],
 [2.5120509548972128, 3],
 [2.564683216305671, 3],
 [2.592778432492835, 3],
 [2.601710975492859, 0],
 [2.8338357397703917, 3],
 [2.892766841624122, 0],
 [2.9394940040762116, 0],
 [3.1742085627759247, 3],
 [3.1934933849939324, 3],
 [3.2758243237389886, 3],
 [3.2797713334926266, 3],
 [3.3504663854454653, 3],
 [3.416580893232297, 0],
 [3.468270462348633, 3],
 [3.4838053906611948, 3],
 [3.5, 3],
 [3.526031763895499, 0],
 [3.5467203160102723, 3],
 [3.555365663331973, 3],
 [3.5676883271945163, 0],
 [3.6117724180795223, 3],
 [3.6224991373359923, 3],
 [3.63890093297413, 3],
 [3.6776656998699595, 3],
 [3.6891733491393426, 3],
 [3.7029852821743705, 0],
 [3.704389288398292, 0],
 [3.711364304403435, 0],
 [3.775980005243672, 0],
 [3.794324709352113, 3],
 [3.877125223667659, 3],
 [3.946846842734083, 0],
 [4.024723592993686, 0

In [97]:
identify=[0 for i in range(len(unique_list))]
for i in range(len(distance)):
  distance[i][1] = int(distance[i][1])
print(distance)
for i in range(num_neighbors):
  identify[distance[i][1]]=identify[distance[i][1]]+1

identify

print("The predicted class is: ",unique_list[identify.index(max(identify))])



[[1.605490579231159, 0], [1.9754746265138408, 3], [2.055632506067172, 0], [2.0796393918177256, 3], [2.304430515333452, 3], [2.316916053723138, 3], [2.466013990227955, 3], [2.5120509548972128, 3], [2.564683216305671, 3], [2.592778432492835, 3], [2.601710975492859, 0], [2.8338357397703917, 3], [2.892766841624122, 0], [2.9394940040762116, 0], [3.1742085627759247, 3], [3.1934933849939324, 3], [3.2758243237389886, 3], [3.2797713334926266, 3], [3.3504663854454653, 3], [3.416580893232297, 0], [3.468270462348633, 3], [3.4838053906611948, 3], [3.5, 3], [3.526031763895499, 0], [3.5467203160102723, 3], [3.555365663331973, 3], [3.5676883271945163, 0], [3.6117724180795223, 3], [3.6224991373359923, 3], [3.63890093297413, 3], [3.6776656998699595, 3], [3.6891733491393426, 3], [3.7029852821743705, 0], [3.704389288398292, 0], [3.711364304403435, 0], [3.775980005243672, 0], [3.794324709352113, 3], [3.877125223667659, 3], [3.946846842734083, 0], [4.024723592993686, 0], [4.052702801834847, 0], [4.053751349